# Examples with run file and C++

This notebooks shows how to **write and execute run files** and **call C++ functions** using the interface from **consav**.

# Setup

In [1]:
%matplotlib inline

# reload module each time cell is run
%load_ext autoreload
%autoreload 2

# use 8 threads in numba
from consav import runtools
runtools.write_numba_config(disable=0,threads=8)

# load the BufferStockModel module
from BufferStockModel import BufferStockModelClass

# Run files

You can create a run file (here **run.py**) calling a specific method (here **test()**), and then run it in a system command.

In [2]:
model = BufferStockModelClass(solmethod='egm')
model.write_run_file(filename='run.py',method='test') # open run.py and have a look
!python run.py

solution time: 0.5 secs
checksum: 3.6669553712300376


This is e.g. beneficial when testing how much speed-up numba provides:

In [3]:
from consav import runtools
model.write_run_file(filename='run.py',solmethod='egm',method='test',Np=1500,Nm=1500,Na=1500)
# here we are making the grids more dense to increase the runtime

for threads in [8,4,2,1]:
    runtools.write_numba_config(threads=threads)
    print(f'threads = {threads}')
    !python run.py
    print('')

threads = 8
solution time: 2.3 secs
checksum: 3.401607450896838

threads = 4
solution time: 4.0 secs
checksum: 3.401607450896838

threads = 2
solution time: 7.5 secs
checksum: 3.401607450896838

threads = 1
solution time: 14.5 secs
checksum: 3.401607450896838



## C++

**Note:** The code below is only for **Windows** and requires installing one of the following two compilers:

* **vs**: Free *Microsoft Visual Studio 2017 Community Edition* ([link](https://visualstudio.microsoft.com/downloads/))
* **intel:** Costly *Intel Parallel Studio 2018 Composer Edition* ([link](https://software.intel.com/en-us/parallel-studio-xe))

**Troubleshooting:** The installation paths for the compiler might need to be adjusted. These are passed as argument to the cpptools.compile() function. Defaults are specified in model.vs_path, model.intel_path, and model.intel_vs_version.

*Additional details are provided in the notebook **Calling C++** in the **Numba and C++** folder.*

### Overview

The **ConsumptionSavingModel** class also have an interface to C++. 

All C++ files should be in **cppfuncs/**. The main file (here **EGM.cpp**) should:

1. Include an **export macro**

  ```
  #define EXPORT extern "C" __declspec(dllexport)
  ```


2. Include the following **struct definitions**:

  ```
  #include "par_struct.cpp"
  #include "sol_struct.cpp"
  #include "sim_struct.cpp"  
  ```
  
  
3. Define **gateway functions** as:

  ```
  EXPORT void myfunction(par_struct *par, sol_struct *sol, sim_struct *sim)
  ```


4. If using visual studio the following function should be included:

  ```
  EXPORT void setup_omp(){
      SetEnvironmentVariable("OMP_WAIT_POLICY", "passive"); 
  }
  ```   

Note, that the structure is such that all C++ functions for simplicity are taking both par, sol and sim as input. The numba types specified in parlist, sollist and simlist are used for writing par_struct.cpp, sol_struc.cpp, and sim_struct.cpp.

### Linking

When initializing a model the **compiler option** can be chosen (defaults to _vs_).

The most import C++ methods are:

1. **setup_cpp()**: Writes *par\_struct.cpp*, *sol\_struct.cpp*, *sim\_strunct.cpp* in cppfuncs/.
* **link_cpp(FILE,FUNCS)**: Link to C++ file **cppfuncs/FILE.cpp** with list of functions in **FUNCS**.
* **call_cpp(FILE,FUNC)**: Call **FUNC** from C++ **FILE**.
* **delink_cpp(FILE)**: Delink C++ library.

### Example

In [4]:
for threads in [8,4,2,1]:
    for compiler in ['vs','intel']:
            
        model = BufferStockModelClass(compiler=compiler,solmethod='egm',Np=1500,Nm=1500,Na=1500)
        print(f'compiler = {compiler}, threads = {threads},')
        model.par.cppthreads = threads
        tic,toc = model.solve_cpp()
        print(f'solved in {toc-tic:.1f} secs')
        model.checksum()
        print('')


compiler = vs, threads = 8,
solved in 3.2 secs
checksum: 3.401607450896838

compiler = intel, threads = 8,
solved in 2.6 secs
checksum: 3.401607450896838

compiler = vs, threads = 4,
solved in 6.4 secs
checksum: 3.401607450896838

compiler = intel, threads = 4,
solved in 5.1 secs
checksum: 3.401607450896838

compiler = vs, threads = 2,
solved in 12.7 secs
checksum: 3.401607450896838

compiler = intel, threads = 2,
solved in 10.0 secs
checksum: 3.401607450896838

compiler = vs, threads = 1,
solved in 25.0 secs
checksum: 3.401607450896838

compiler = intel, threads = 1,
solved in 19.4 secs
checksum: 3.401607450896838



### Installation paths

In [5]:
model = BufferStockModelClass(compiler=compiler,solmethod='egm')
print(model.vs_path)
print(model.intel_path)
print(model.intel_vs_version)

C:/Program Files (x86)/Microsoft Visual Studio/2017/Community/VC/Auxiliary/Build/
C:/Program Files (x86)/IntelSWTools/compilers_and_libraries_2018.5.274/windows/bin/
vs2017
